In [2]:
!pip install arxiv

     |████████████████████████████████| 81 kB 4.4 MB/s eta 0:00:01
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
    Running setup.py install for sgmllib3k ... done
You should consider upgrading via the '/Users/apple/Desktop/LangChain/.venv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [11]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=100)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [12]:
wiki.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OllamaEmbeddings(model="phi")
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embedding=embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123a39dc0>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any question about LangSmith, you must use this tool.")

In [10]:
retrieval_tool.name

'langsmith_search'

In [13]:
tools = [wiki,retrieval_tool]

In [14]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi")

/var/folders/8z/lfg_cf355xj5jgmz32jg4kpm0000gn/T/ipykernel_43693/3631292146.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi")


In [15]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [27]:
##Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [31]:
result=agent_executor.invoke({"input" : "Tell me about LangSmith"})
print(result)



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output

In [34]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import Ollama
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# -------------------------
# LLM (local, stable)
# -------------------------
llm = Ollama(
    model="phi",        # you can switch to "mistral" later
    timeout=120
)

# -------------------------
# Tool: Wikipedia
# -------------------------
wiki_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper()
)
tools = [wiki_tool]

# -------------------------
# ReAct Prompt (LLM-agnostic)
# -------------------------
prompt = hub.pull("hwchase17/react")

# -------------------------
# Agent (ReAct style)
# -------------------------
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# -------------------------
# Agent Executor (IMPORTANT FLAGS)
# -------------------------
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,   # ⭐ fixes your last error
    max_iterations=5
)

# -------------------------
# Invoke (ONLY valid input key)
# -------------------------
result = agent_executor.invoke(
    {"input": "Tell me about LangSmith"}
)

print("\nFINAL ANSWER:\n")
print(result["output"])




> Entering new AgentExecutor chain...
 Question: Tell me about LangSmith.

The article Wikipedia provides information on LangSmith. A brief summary is as follows: 

LangSmith (also known as L&S or L and S) was a short-lived American software company founded in 2000 by Larry Smith, Chris Langman, and Paul Sabin. The company's products were an attempt to combine the best of both Microsoft Office and Apple's Macromedia Flash. They released two products, MSL (Microsoft Live) for Windows 95/98 and MSXML for Windows NT 4.0.

In a nutshell, the goal was to take a Microsoft Word document, translate it into something that would run in an Adobe Flash browser window, which could then be viewed with no special program or plugin installed. This meant that users of Macs running Windows would have the same experience as those using PCs running Windows. 

Their products were not well received by either side, and they ultimately failed to make a lasting impression on the market. MSL for Windows 95/98